<a href="https://colab.research.google.com/github/bishwajit02/Pytorch-Tutorial-Youtube/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simple Neural Network using Iris Dataset**

In [1]:
import torch
import torch.nn as nn # nn stands for neural network
import torch.nn.functional as F

In [3]:
# Create a model class that inherits nn.Module

class Model(nn.Module):
  # Input Layers (4 features)
  # --> Then to hidden layers --> Output (3 classes of Iris flower)

# The 'def __init__' function sets up the class
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__()  # Instantiate the model
    self.fc1 = nn.Linear(in_features, h1)   #fc - Fully Connected (Important for Neural Network)
    self.fc2 = nn.Linear(h1, h2)  # The previous result h1 is now connected with the new layer h2
    self.out = nn.Linear(h2, out_features)  # h2 is the last layer so it's directly connected to the output

  def forward(self, x):
    x = F.relu(self.fc1(x))  # relu - Rectified Linear Unit, More - https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU,
    # If the input is positive (greater than zero): ReLU lets the input pass through unchanged.
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [4]:
# Pick a manual seed for randomization - helps with reproductibility
torch.manual_seed(42)

# Create an instance for the model

model = Model()